In [1]:
from keras import backend as bknd
from keras.callbacks import *
from keras.layers import *
from keras.models import *
from keras.optimizers import SGD
from keras.utils import *

from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
#from stn import spatial_transformer_network as transformer

import numpy as np
import keras
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten,concatenate 
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import metrics

from sklearn.utils import class_weight
from collections import Counter

import matplotlib.pyplot as plt

from os import listdir
from os.path import isfile, join

import pandas as pd
#from STN.spatial_transformer import SpatialTransformer

In [2]:
# help expand


In [3]:
# spec_train_path = '../input/modified-mfcc-40/train/train'
# # spec_train_path_0db = '/content/gdrive/MyDrive/Spectogram7000_0db/train'
# spec_val_path = '../input/modified-mfcc-40/val/val_0'

In [4]:
import keras,os
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [5]:
# VGG.input

In [6]:
# model = keras.models.Model(inputs = VGG.input,outputs = prediction)

In [7]:
# model.summary()

In [8]:
# from keras.callbacks import ModelCheckpoint
# from keras.callbacks import EarlyStopping
# filepath="../working/best_model1.h5"
# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
# es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=50)
# callbacks_list = [checkpoint,es]

In [9]:
# model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00001),loss=keras.losses.categorical_crossentropy,metrics=['accuracy'])

In [10]:
# hist = model.fit_generator(generator=train_data,epochs=100,validation_data=test_data,callbacks=callbacks_list)

In [13]:
train_loc = 'CWT\\train_4000'
test_loc = 'CWT\\clean\\test'
inputShape = Input((224,224, 3))  # base on Tensorflow backend
conv_1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputShape)
x_inputi = BatchNormalization()(conv_1)
conv_2 = Conv2D(64, (3, 3), activation='relu', padding='same')(x_inputi)
conv_3 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv_2)
batchnorm_3 = BatchNormalization()(conv_3)
x_inputi= MaxPooling2D(pool_size=(2, 2))(batchnorm_3)
conv_4 = Conv2D(64, (3, 3), activation='relu', padding='same')(x_inputi)
conv_5 = Conv2D(96, (5, 5), activation='relu', padding='same')(conv_4)
batchnorm_5 = BatchNormalization()(conv_5)
x_inputi= MaxPooling2D(pool_size=(2, 2))(batchnorm_5)
x_in1i = Conv2D(filters = int(x_inputi.shape[-1]//4), kernel_size = (1,1), strides = (1,1), padding = 'valid', activation = 'relu')(x_inputi)
x_in1i=BatchNormalization()(x_in1i)
x_in2i = Conv2D(filters = int(x_inputi.shape[-1]//2), kernel_size = (1,1), strides = (1,1), padding = 'valid', activation = 'relu')(x_in1i)
x_in2i=BatchNormalization()(x_in2i)
x_in3i = Conv2D(filters = int(x_inputi.shape[-1]//2), kernel_size = (3,3), strides = (1,1), padding = 'same', activation = 'relu')(x_in1i)
x_in3i=BatchNormalization()(x_in3i)
out = Concatenate() ([x_in2i, x_in3i])
conv_6 = Conv2D(96, (5, 5), activation='relu', padding='same')(out)
conv_7 = Conv2D(96, (5, 5), activation='relu', padding='same')(conv_6)
batchnorm_7 = BatchNormalization()(conv_7)
#pool_5 = MaxPooling2D(pool_size=(2, 2))(batchnorm_7)
bn_shape = batchnorm_7.get_shape()
print(bn_shape)  # (?, 50, 7, 512)

# reshape to (batch_size, width, height*dim)
# x_reshape = Reshape(target_shape=(int(bn_shape[1]), int(bn_shape[2] * bn_shape[3])))(stn_7)
x_reshape = Reshape(target_shape=(int(bn_shape[1]), int(bn_shape[2] * bn_shape[3])))(batchnorm_7)

fc_1 = Dense(128, activation='relu')(x_reshape)  # (?, 50, 128)

print(x_reshape.get_shape())  # (?, 50, 3584)
print(fc_1.get_shape())  # (?, 50, 128)

rnn_1 = LSTM(32, kernel_initializer="he_normal", return_sequences=True)(fc_1)
rnn_1b = LSTM(32, kernel_initializer="he_normal", go_backwards=True, return_sequences=True)(fc_1)
rnn1_merged = add([rnn_1, rnn_1b])

rnn_2 = LSTM(32, kernel_initializer="he_normal", return_sequences=True)(rnn1_merged)
rnn_2b = LSTM(32, kernel_initializer="he_normal", go_backwards=True, return_sequences=True)(rnn1_merged)
rnn2_merged = concatenate([rnn_2, rnn_2b])

drop_1 = Dropout(0.25)(rnn2_merged)
classifier=Flatten()(drop_1)
#fc_2 = Dense(512, kernel_initializer='he_normal', activation='relu')(classifier)
fc_2 = Dense(256, kernel_initializer='he_normal', activation='relu')(classifier)
fc_2 = Dense(128, kernel_initializer='he_normal', activation='relu')(classifier)
fc_2 = Dense(64, kernel_initializer='he_normal', activation='relu')(fc_2)
fc_2 = Dense(32, kernel_initializer='he_normal', activation='relu')(fc_2)
fc_2 = Dense(16, kernel_initializer='he_normal', activation='relu')(fc_2)
fc_2 = Dense(5, kernel_initializer='he_normal', activation='softmax')(fc_2)

# model setting
base_model = Model(inputs=inputShape, outputs=fc_2) 
adam = keras.optimizers.Adam(lr = 0.0001)
base_model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
trdata = ImageDataGenerator(rescale=1./255)
traindata = trdata.flow_from_directory(directory=train_loc, target_size=(224,224),batch_size=16)
tsdata = ImageDataGenerator(rescale=1./255)
testdata = tsdata.flow_from_directory(directory=test_loc, target_size=(224,224),batch_size=16)
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# checkpoint1 = ModelCheckpoint("CWT//cwt_weights//model_0_cwt_v2_awgn.h5", monitor='val_accuracy', verbose=1, 
#                              save_best_only=True, save_weights_only=False, mode='auto')


# STEP_SIZE_TRAIN=traindata.n//traindata.batch_size
# STEP_SIZE_VALID=testdata.n//testdata.batch_size
# from collections import Counter
# counter = Counter(traindata.classes)                       
# max_val = float(max(counter.values()))   
# class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}
# class_weights
from keras.callbacks import EarlyStopping
#early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')
#checkpoint = ModelCheckpoint("working/best_model.h5", monitor='val_accuracy', verbose=1, 
   #                          save_best_only=True, save_weights_only=False, mode='auto')
# early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')
# history = base_model.fit_generator(traindata,
#                     epochs =60,
#                     steps_per_epoch=STEP_SIZE_TRAIN,
#                     class_weight=class_weights,
#                     validation_data =testdata,
#                     validation_steps=STEP_SIZE_VALID,
#                     callbacks=[checkpoint1]
#                     )

filepath="CWT//cwt_weights//model_0_cwt_v2_awgn_trial.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=20)
callbacks_list = [checkpoint,es]
base_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),loss=keras.losses.categorical_crossentropy,metrics=['accuracy'])
hist = base_model.fit_generator(epochs=60,generator=traindata,validation_data=testdata,callbacks=callbacks_list)

(None, 56, 56, 96)
(None, 56, 5376)
(None, 56, 128)


C:\ProgramData\Anaconda3\envs\py37gpu\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Found 4000 images belonging to 5 classes.
Found 200 images belonging to 5 classes.
Epoch 1/60


C:\ProgramData\Anaconda3\envs\py37gpu\lib\site-packages\ipykernel_launcher.py:95: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


250/250 [==============================] - ETA: 0s - loss: 1.2468 - accuracy: 0.4815
Epoch 1: val_accuracy improved from -inf to 0.20000, saving model to CWT//cwt_weights\model_0_cwt_v2_awgn_trial.h5
250/250 [==============================] - 38s 108ms/step - loss: 1.2468 - accuracy: 0.4815 - val_loss: 1.7808 - val_accuracy: 0.2000
Epoch 2/60
250/250 [==============================] - ETA: 0s - loss: 0.6220 - accuracy: 0.7703
Epoch 2: val_accuracy improved from 0.20000 to 0.82500, saving model to CWT//cwt_weights\model_0_cwt_v2_awgn_trial.h5
250/250 [==============================] - 25s 100ms/step - loss: 0.6220 - accuracy: 0.7703 - val_loss: 0.5568 - val_accuracy: 0.8250
Epoch 3/60
250/250 [==============================] - ETA: 0s - loss: 0.4276 - accuracy: 0.8403
Epoch 3: val_accuracy improved from 0.82500 to 0.95500, saving model to CWT//cwt_weights\model_0_cwt_v2_awgn_trial.h5
250/250 [==============================] - 25s 99ms/step - loss: 0.4276 - accuracy: 0.8403 - val_loss: 0

Epoch 29/60
250/250 [==============================] - ETA: 0s - loss: 2.4578e-04 - accuracy: 1.0000
Epoch 29: val_accuracy did not improve from 0.97500
250/250 [==============================] - 23s 94ms/step - loss: 2.4578e-04 - accuracy: 1.0000 - val_loss: 0.1820 - val_accuracy: 0.9750
Epoch 30/60
250/250 [==============================] - ETA: 0s - loss: 1.4080e-04 - accuracy: 1.0000
Epoch 30: val_accuracy did not improve from 0.97500
250/250 [==============================] - 24s 96ms/step - loss: 1.4080e-04 - accuracy: 1.0000 - val_loss: 0.1851 - val_accuracy: 0.9750
Epoch 31/60
250/250 [==============================] - ETA: 0s - loss: 1.3943e-04 - accuracy: 1.0000
Epoch 31: val_accuracy did not improve from 0.97500
250/250 [==============================] - 25s 99ms/step - loss: 1.3943e-04 - accuracy: 1.0000 - val_loss: 0.1948 - val_accuracy: 0.9750
Epoch 32/60
250/250 [==============================] - ETA: 0s - loss: 1.2021e-04 - accuracy: 1.0000
Epoch 32: val_accuracy did no

In [14]:
model = keras.models.load_model("CWT//cwt_weights//model_0_cwt_v2_awgn_trial.h5")

In [17]:
hisDF=pd.DataFrame(hist.history)
hisDF.to_csv('CWT//history_model_0_cwt_v2_trial.csv')

In [18]:
spec_test_path = 'CWT//clean//test//'
spec_test_path1 = 'CWT//0dB//test//'
spec_test_path2= 'CWT//5dB//test//'
spec_test_path3= 'CWT//10dB//test//'
spec_test_path4= 'CWT//15dB//test//'
test_gen = ImageDataGenerator(rescale=1./255)
test_gen1 = ImageDataGenerator(rescale=1./255)
test_gen2 = ImageDataGenerator(rescale=1./255)
test_gen3 = ImageDataGenerator(rescale=1./255)
test_gen4 = ImageDataGenerator(rescale=1./255)
test = test_gen.flow_from_directory(directory=spec_test_path,target_size=(224,224),batch_size=16,shuffle=False)
test.reset()
test1 = test_gen1.flow_from_directory(directory=spec_test_path1,target_size=(224,224),batch_size=16,shuffle=False)
test1.reset()
test2 = test_gen2.flow_from_directory(directory=spec_test_path2,target_size=(224,224),batch_size=16,shuffle=False)
test2.reset()
test3 = test_gen3.flow_from_directory(directory=spec_test_path3,target_size=(224,224),batch_size=16,shuffle=False)
test3.reset()
test4 = test_gen4.flow_from_directory(directory=spec_test_path4,target_size=(224,224),batch_size=16,shuffle=False)
test4.reset()

Found 200 images belonging to 5 classes.
Found 200 images belonging to 5 classes.
Found 200 images belonging to 5 classes.
Found 200 images belonging to 5 classes.
Found 200 images belonging to 5 classes.


In [19]:
scoreSeg = model.evaluate_generator(test)
print("Accuracy clean= ",scoreSeg[1])
scoreSeg = model.evaluate_generator(test1)
print("Accuracy 0dB= ",scoreSeg[1])
scoreSeg = model.evaluate_generator(test2)
print("Accuracy 5dB= ",scoreSeg[1])
scoreSeg = model.evaluate_generator(test3)
print("Accuracy 10dB= ",scoreSeg[1])
scoreSeg = model.evaluate_generator(test4)
print("Accuracy 15dB= ",scoreSeg[1])

C:\ProgramData\Anaconda3\envs\py37gpu\lib\site-packages\ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


Accuracy clean=  0.9750000238418579


C:\ProgramData\Anaconda3\envs\py37gpu\lib\site-packages\ipykernel_launcher.py:3: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


Accuracy 0dB=  0.48500001430511475


C:\ProgramData\Anaconda3\envs\py37gpu\lib\site-packages\ipykernel_launcher.py:5: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """


Accuracy 5dB=  0.7350000143051147


C:\ProgramData\Anaconda3\envs\py37gpu\lib\site-packages\ipykernel_launcher.py:7: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  import sys


Accuracy 10dB=  0.8849999904632568


C:\ProgramData\Anaconda3\envs\py37gpu\lib\site-packages\ipykernel_launcher.py:9: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  if __name__ == "__main__":


Accuracy 15dB=  0.9350000023841858


spec_test_path = '../input/combined-v1/combined/val'

In [ ]:
model = keras.models.load_model("CWT//cwt_weights//best_model1_kutta_awgn.h5")

In [ ]:
scoreSeg = model.evaluate_generator(test)
print("Accuracy = ",scoreSeg[1])


In [ ]:
scoreSeg

In [ ]:
predicted_class_indices=np.argmax(scoreSeg[1],axis=1)

In [ ]:
predicted_class_indices

In [ ]:

from sklearn.metrics import classification_report, confusion_matrix

print('Confusion Matrix')
print(confusion_matrix(test.classes, predicted_class_indices))
print('Classification Report')

print(classification_report(test.classes, predicted_class_indices))#, target_names=target_names))

In [ ]:
import pandas as pd

In [ ]:
cl = np.round(scoreSeg)

filenames=test.filenames

In [ ]:
results=pd.DataFrame({"file":filenames,"pr":pred[:,0], "class":cl[:,0]})

In [ ]:
results

In [ ]:
from IPython.display import FileLink
FileLink(r'./weights.best.hdf5')